# Setup

Import a few common modules

In [2]:
#CODE for points = 1
# import sklearn, numpy, os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import sklearn
import numpy as np
import os

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [3]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [4]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [9]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

def relu(z):
    return np.maximum(z, 0)
initializer = tf.keras.initializers.HeNormal()
keras.layers.Dense(10, activation= relu, kernel_initializer= initializer)

### Leaky ReLU

In [11]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha= 0.01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [13]:
#CODE for points = 1
#load MNIST dataset from keras
from keras.datasets import mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [17]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer= initializer),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer= initializer),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation= "softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5339 - accuracy: 0.8223 - val_loss: 0.5156 - val_accuracy:

Now look at what happens if we try to use the ReLU activation function instead:

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [20]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.8006 - accuracy: 0.8930 - val_loss: 0.7832 - val_accuracy: 0.8996
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7930 - accuracy: 0.8948 - val_loss: 0.7603 - val_accuracy: 0.9082
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7860 - accuracy: 0.8984 - val_loss: 0.7591 - val_accuracy: 0.9070
Epoch 4/5
1141/1719 [==================>...........] - ETA: 3s - loss: 0.7809 - accuracy: 0.9020

Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [22]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_113 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_114 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [24]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
#bn1.updates #deprecated

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [26]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8619 - accuracy: 0.7335 - val_loss: 0.4687 - val_accuracy: 0.8716
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4346 - accuracy: 0.8734 - val_loss: 0.3494 - val_accuracy: 0.9038
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3559 - accuracy: 0.8965 - val_loss: 0.3001 - val_accuracy: 0.9174
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3115 - accuracy: 0.9101 - val_loss: 0.2695 - val_accuracy: 0.9242
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2815 - accuracy: 0.9175 - val_loss: 0.2444 - val_accuracy: 0.9306


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [28]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [29]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 8s 4ms/step - loss: 1.1288 - accuracy: 0.6813 - val_loss: 0.6171 - val_accuracy: 0.8486
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5778 - accuracy: 0.8548 - val_loss: 0.4354 - val_accuracy: 0.8920
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4590 - accuracy: 0.8818 - val_loss: 0.3585 - val_accuracy: 0.9108
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3954 - accuracy: 0.8957 - val_loss: 0.3152 - val_accuracy: 0.9184
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3552 - accuracy: 0.9033 - val_loss: 0.2840 - val_accuracy: 0.9252


# Faster Optimizers

## Momentum optimization

In [30]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate= 0.1, momentum= 0.0)

## Nesterov Accelerated Gradient

In [32]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate= 0.1, momentum= 0.8, nesterov= True)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [45]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

lr2 = tf.keras.regularizers.l2(l2=0.01)
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer= lr2)

In [51]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer
from tensorflow.keras.optimizers import Nadam
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer= lr2),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer= lr2),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer= lr)
])

   
model.compile(loss="sparse_categorical_crossentropy", optimizer= "nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 1.2753 - accuracy: 0.8758 - val_loss: 0.8178 - val_accuracy: 0.8878
Epoch 2/2
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8215 - accuracy: 0.8837 - val_loss: 0.7774 - val_accuracy: 0.8984


In [52]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer= lr2)

In [54]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation= "softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer= "nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 1.2832 - accuracy: 0.8766 - val_loss: 0.8188 - val_accuracy: 0.8892
Epoch 2/2
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8229 - accuracy: 0.8842 - val_loss: 0.7778 - val_accuracy: 0.9014
